In [16]:
import numpy as np
from numpy import newaxis

# Rastrigin-Funktion
def f(x):
    n = len(x)
    return 10*n + np.dot(x, x) - 10*np.sum(np.cos(2*np.pi*x))

In [161]:
# (mu, lambda)-ES:
def es_optimization(f, x0_pop, lam, mu, kappa, sigma, max_stuck_trials):
    P = x0_pop # R^(muxn) eine Reihe pro Individuum
    max_stuck_trials = max_stuck_trials # number of iterations to do after not improving on best result
    stuck_trials = 0
    status_quo_best_f = [np.inf]
    status_quo_best_x = x0_pop[0, :]
    n_it = 0

    stop = False
    while not stop:
        n_it += 1
        E = P
        C = variation(reproduktion(E), sigma, lam)
        assert C.shape == (lam, n)

        P, best_f, best_x = selektion(f, kappa, E, C)
        
        print(f"Best f={best_f} at x={best_x}")

        if best_f < status_quo_best_f:
            stuck_trials = 0
            status_quo_best_f, status_quo_best_x = best_f, best_x
        else:
            stuck_trials += 1
        if stuck_trials == max_stuck_trials: stop = True
        
    print(f"Final best f={status_quo_best_f} at x={status_quo_best_x}")
    print(f"# iterations: {n_it}")


def variation(E, sigma, lam):
    # mu * n * lam
    dim = int(lam/E.shape[0])

    variations_matrix = np.random.normal(loc=0, scale=sigma, size=E.shape[0]*E.shape[1]*dim).reshape((dim, E.shape[0], E.shape[1]))
    E = E[newaxis, :, :]
    C = (variations_matrix + E).reshape(E.shape[1]*dim, E.shape[2])
    return C[:lam, :]


def reproduktion(E):
    return E


def selektion(f, kappa, E, C):
    if kappa == 1:
        P = C
    elif kappa == np.inf:
        P = np.concatenate([E, C], axis=0)
    else:
        raise NotImplementedError

    mu = E.shape[0]
    funktionsauswertungen = np.apply_along_axis(f, 1, P)
    idx_kleinste = np.argpartition(funktionsauswertungen, mu)[:mu]

    best_f, best_x = np.min(funktionsauswertungen), P[np.argmin(funktionsauswertungen), :]

    P = P[idx_kleinste, :]
    return P, best_f, best_x


In [162]:
lam = 500
mu = 10
kappa = 1
sigma = 0.2
n = 5
x0_pop = np.random.normal(loc=0, scale=1, size=n*mu).reshape((mu, n))
#x0_pop = np.zeros((mu, n)) + 5
max_stuck_trials = 20

In [163]:
es_optimization(f, x0_pop, lam, mu, kappa, sigma, max_stuck_trials)

Best f=10.04119711892833 at x=[ 1.02393873  0.12336986 -0.12094969  0.09984    -0.95381095]
Best f=3.7587676942216675 at x=[ 0.98974877  0.06211665 -0.07171228 -0.03428184  0.06258183]
Best f=3.3417559851834895 at x=[ 1.03994089  0.04079227 -0.02356347 -0.04107438 -0.96467687]
Best f=2.5558494458775343 at x=[ 1.07403307e+00 -3.88366836e-02 -3.94689471e-03 -1.39816409e-02
 -9.47781027e-04]
Best f=4.862251159696598 at x=[ 1.0025255   0.04336687  0.0578396   0.0577812  -0.91688796]
Best f=3.5617343056802184 at x=[1.07722994 0.06377486 0.04458813 0.01642755 0.00517704]
Best f=3.576144795122417 at x=[ 1.06826188 -0.00158324  0.05859023  0.05549122  0.03560742]
Best f=6.9626202819761716 at x=[ 0.94395608  0.1062002  -0.11366856 -0.06519854 -0.00820906]
Best f=3.8742168690721854 at x=[ 1.04266193  0.03612122  0.07380905  0.03870988 -0.06452173]
Best f=4.34569033431039 at x=[ 0.92856162  0.03121584 -0.06651176 -0.02812341 -1.0328249 ]
Best f=3.111945428675199 at x=[ 0.94168211  0.02522866 -0.0